In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy


In [25]:
# Example usage
user_ingredients = ["vaj", "fokhagyma", "liszt"]
query = "Egy sajtos levest szeretnék enni, amiben még a fokhagyma dominál"
user_recipe_id = 3

In [3]:
df = pd.read_json("recipes.json")

df['ingredients_str'] = df['ingredients'].apply(lambda x: " ".join(x))

# Check the preprocessed data
print(df[['title', 'ingredients_str']].head())

,id,title,content,ingredients
0,0,Abált szalonna,Frissen bontott tokaszalonnát sós vízben forrá...,"[tokaszalonna, só, fokhagyma, fűszerpaprika, f..."
1,1,Áfonya torta,A makadámdiót ledaráljuk. A tésztához a szójat...,"[tönköly búzaliszt, makadámdió, szójatejföl, n..."
2,2,Áfonyás húsgolyó,25 dkg kukoricadarát annyi sós vízben megfőzün...,[]
3,3,"Áfonyás, kevert sütemény mandulával",A sütőt 190 fokra előmelegítjük. Egy 20x30 cm-...,"[a morzsához:, 40 g liszt, 100 g cukor, 1 tk ő..."
4,4,Áfonyás-narancsos sajttorta,"A kekszet, a zabpelyhet és a nádcukrot összeda...","[5 dk teljes kiőrlésű búzadarás keksz, 5 dkg z..."
...,...,...,...,...
4917,4917,Zöldségleves csipetkével és csirkével,"A zöldségeket megmosom, kockára vágom. A hagym...","[1/2 csirke,, 1 kg burgonya,, 3 új sárgarépa,,..."
4918,4918,Zsidó tojás,A hagymát apróra vágjuk és serpenyőben megpirí...,"[0 .5 kg kacsamáj, 2 db tojás, mustár, 1 nagy ..."
4919,4919,Zöld almás görögdinnye frissítő,Egyszerű dolgunk van. A kimagozott görögdinnye...,"[7,5 dl görögdinnyevelő, 2 db zöld alma, 20 sz..."
4920,4920,Zserbó,A tésztához felfuttatjuk az élesztőt. A liszte...,"[50 dkg liszt, 10 dkg cukor, 20 dkg zsír, 1,5 ..."


In [9]:
vectorizer = TfidfVectorizer()
ingredient_vectors = vectorizer.fit_transform(df['ingredients_str'])

# Find similar recipes
similarity_matrix = cosine_similarity(ingredient_vectors)

In [10]:
def recommend_recipe(recipe_id, similarity_matrix, df, top_n=5):
    sim_scores = list(enumerate(similarity_matrix[recipe_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_recipes = [df.iloc[i[0]] for i in sim_scores[1:top_n+1]]
    return top_recipes

In [26]:
# Find similar recipes to the recipe that has 3 as ID
recommend_recipe(user_recipe_id, similarity_matrix, df)

[id                                                                     4590
 title                                   Áfonyás, kevert sütemény mandulával
 content                   A sütőt 190 fokra előmelegítjük. Egy 20x30 cm-...
 ingredients               [a morzsához:, 40 g liszt, 100 g cukor, 1 tk ő...
 ingredients_str           a morzsához: 40 g liszt 100 g cukor 1 tk őrölt...
 ingredients_normalized    [a morzsához:, 40 g liszt, 100 g cukor, 1 tk ő...
 ingredient_overlap                                                        0
 Name: 4590, dtype: object,
 id                                                                     3588
 title                                                   Céklás-almás morzsa
 content                   Egy keverőtálban összekeverünk mandulalisztet,...
 ingredients               [50 g pekándió, 50 g fenyőmag, 150 g cékla, ba...
 ingredients_str           50 g pekándió 50 g fenyőmag 150 g cékla balzsa...
 ingredients_normalized    [50 g pekándió, 50 g 

In [12]:
# Normalize ingredients (convert to lowercase for consistency)
df['ingredients_normalized'] = df['ingredients'].apply(
    lambda x: [ingredient.lower() for ingredient in x]
)

# Check the normalized data
print(df[['title', 'ingredients_normalized']].head())

                                 title  \
0                       Abált szalonna   
1                         Áfonya torta   
2                     Áfonyás húsgolyó   
3  Áfonyás, kevert sütemény mandulával   
4          Áfonyás-narancsos sajttorta   

                              ingredients_normalized  
0  [tokaszalonna, só, fokhagyma, fűszerpaprika, f...  
1  [tönköly búzaliszt, makadámdió, szójatejföl, n...  
2                                                 []  
3  [a morzsához:, 40 g liszt, 100 g cukor, 1 tk ő...  
4  [5 dk teljes kiőrlésű búzadarás keksz, 5 dkg z...  


In [27]:
def find_recipes_by_ingredients(user_ingredients, data, top_n=5):
    user_ingredients = [ingredient.lower() for ingredient in user_ingredients]  # Normalize user input
    
    # Calculate overlap score for each recipe
    data['ingredient_overlap'] = data['ingredients_normalized'].apply(
        lambda recipe_ingredients: len(set(user_ingredients) & set(recipe_ingredients))
    )
    
    # Sort recipes by overlap score in descending order
    recommendations = data.sort_values(by='ingredient_overlap', ascending=False).head(top_n)
    
    # Return the top matching recipes
    return recommendations[['id', 'title', 'ingredient_overlap']]

print(find_recipes_by_ingredients(user_ingredients, df))

        id                                              title  \
2409  2409                       Radicchios-mentás palacsinta   
2338  2338                            Pisztráng májjal töltve   
2831  2831                                  Sütőtökös rizottó   
2627  2627         Scampikrémmel töltött tintahal brokkolival   
328    328  Borjútekercs marsalaval ízesített kacsamájjal ...   

      ingredient_overlap  
2409                   3  
2338                   3  
2831                   3  
2627                   3  
328                    3  


In [28]:
def combined_recommendation(user_ingredients, similarity_matrix, data, top_n=5):
    # Match by ingredients
    ingredient_matches = find_recipes_by_ingredients(user_ingredients, data, top_n=None)
    
    # Add similarity scores
    ingredient_matches['similarity_score'] = ingredient_matches['id'].apply(
        lambda recipe_id: similarity_matrix[data.index[data['id'] == recipe_id][0]].mean()
    )
    
    # Combine scores (weighted average)
    ingredient_matches['combined_score'] = ingredient_matches['ingredient_overlap'] * 0.7 + ingredient_matches['similarity_score'] * 0.3
    
    # Sort by combined score
    recommendations = ingredient_matches.sort_values(by='combined_score', ascending=False).head(top_n)
    
    return recommendations[['id', 'title', 'combined_score']]

# Example usage
print(combined_recommendation(user_ingredients, similarity_matrix, df))


        id                                              title  combined_score
2831  2831                                  Sütőtökös rizottó        2.111917
2338  2338                            Pisztráng májjal töltve        2.111856
2409  2409                       Radicchios-mentás palacsinta        2.111794
2627  2627         Scampikrémmel töltött tintahal brokkolival        2.110999
328    328  Borjútekercs marsalaval ízesített kacsamájjal ...        2.110577


In [17]:
# Load spaCy's NLP model
nlp = spacy.load("en_core_web_sm")

# Example recipe data
data = pd.read_json("recipes.json")

# Normalize ingredients and combine features
data['ingredients_normalized'] = data['ingredients'].apply(lambda x: [ing.lower() for ing in x])
data['ingredients_str'] = data['ingredients_normalized'].apply(lambda x: " ".join(x))
data['combined_features'] = data['ingredients_str'] + " " + data['content']

# Vectorize the combined features for similarity search
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['combined_features'])


In [22]:
def recommend_recipes(query, data, tfidf_matrix, top_n=5):
    """
    Recommend recipes based on free text or ingredient list query.
    """
    if isinstance(query, list):  # If input is a list of ingredients
        query = " ".join([q.lower() for q in query])  # Join ingredients into a single string
    
    # Use spaCy to process free-text queries
    doc = nlp(query.lower())
    keywords = " ".join([token.text for token in doc if token.is_alpha])  # Extract meaningful words
    
    # Transform the query into a TF-IDF vector
    query_vector = vectorizer.transform([keywords])
    
    # Calculate cosine similarity with the recipe matrix
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Sort recipes by similarity score
    data['similarity_score'] = similarities
    recommendations = data.sort_values(by='similarity_score', ascending=False).head(top_n)
    
    return recommendations[['id', 'title', 'similarity_score']]

# Example 1: Free-text query
query_text = "sajtos leves fokhagmával"
print(recommend_recipes(query_text, data, tfidf_matrix))

# Example 2: Ingredient list
query_ingredients = ["sajt", "bacon", "csirke"]
print(recommend_recipes(query_ingredients, data, tfidf_matrix))


        id                                              title  \
3279  3279                                    Ünnepi borleves   
3264  3264                            Tyúkhúsleves - Balástya   
3216  3216  „Torta” burgonyából és taleggio sajtból, debre...   
3263  3263                                Tyúkhúsleves - Baks   
2277  2277                          Parmezános fokhagymaleves   

      similarity_score  
3279          0.200471  
3264          0.167352  
3216          0.159498  
3263          0.156522  
2277          0.153987  
        id                                              title  \
4495  4495  Puliszkás millefeuille olvasztott sajttal és b...   
2857  2857                               Szárd töltött csirke   
3211  3211                   Török csirke cseréptálban - Győr   
3818  3818                                   Házi hamburgerek   
1458  1458              Karády Katalin saláta - Nyár van menü   

      similarity_score  
4495          0.290298  
2857          0.22

In [29]:
def combined_scoring(query, data, tfidf_matrix, top_n=5):
    # Calculate text similarity
    recommendations = recommend_recipes(query, data, tfidf_matrix, top_n=None)
    
    # Calculate ingredient overlap for ingredient-based queries
    if isinstance(query, list):
        query_set = set([q.lower() for q in query])
        data['ingredient_overlap'] = data['ingredients_normalized'].apply(
            lambda x: len(query_set & set(x))
        )
    else:
        data['ingredient_overlap'] = 0
    
    # Combine similarity and overlap scores
    data['combined_score'] = data['similarity_score'] * 0.7 + data['ingredient_overlap'] * 0.3
    recommendations = data.sort_values(by='combined_score', ascending=False).head(top_n)
    
    return recommendations[['id', 'title', 'combined_score']]

# Example with combined scoring
print(combined_scoring(query_ingredients, data, tfidf_matrix))


        id                              title  combined_score
3440  3440  Zöldsaláta balzsamos gombaraguval        0.435618
3403  3403              Zacskóban sült csirke        0.423591
4834  4834                             Quiche        0.370622
4503  4503                             Quiche        0.370622
4724  4724              Juhtúrós borkorcsolya        0.367814
